In [1]:
from detector_map import *
import glob

## Silicon Assembly

In [2]:
#SMuRF channel assignments
chan_assign=glob.glob("Sv5/*channel_assignment*.txt")
chan_assign.sort()
N_chan=chan_assign[4:] 
S_chan=chan_assign[:4]
highband='S' #Which side is 6-8 GHz
dark_bias_lines=[] #If certain sides are covered

In [7]:
N_vna_map="Sv5/N_vna_map.csv"
S_vna_map="Sv5/S_vna_map.csv"
posnfile="Sv5/band2posn.csv"
design_file="metadata/umux_32_map.pkl"
waferfile="metadata/UFM_Si_corrected.csv"

In [4]:
#South side VNA match
S_smurf2freq=all_smurf_chan_to_realized_freq(S_chan)
S_vna2pad=vna_freq_to_muxpad(S_vna_map,design_file)
S_smurf2mux=smurf_to_mux(S_smurf2freq,S_vna2pad,threshold=0.01)

#Repeat for north side VNA. 
N_smurf2freq=all_smurf_chan_to_realized_freq(N_chan)
N_vna2pad=vna_freq_to_muxpad(N_vna_map,design_file)
N_smurf2mux=smurf_to_mux(N_smurf2freq,N_vna2pad,threshold=0.01)

#Put things together
smurf2mux=pd.concat([S_smurf2mux,N_smurf2mux],axis=0)

In [5]:
#All Smurf frequency to mux bondpad assignments
smurf2padloc=mux_band_to_mux_posn(smurf2mux,posnfile,highband='S')
smurf2padloc

,smurf_band,smurf_chan,smurf_freq,mux_band,pad,index,design_freq,vna_freq,mux_posn
0,4.0,271.0,4217.078,1.0,-1.0,0.0,4152.428571,4217.078,16.0
1,4.0,335.0,4217.568,1.0,3.0,1.0,4154.299286,4217.568,16.0
2,4.0,399.0,4218.468,1.0,7.0,2.0,4156.170000,4218.468,16.0
3,4.0,47.0,4219.630,1.0,11.0,3.0,4158.040714,4219.630,16.0
4,4.0,111.0,4220.800,1.0,19.0,5.0,4161.782143,4220.800,16.0
...,...,...,...,...,...,...,...,...,...
1129,3.0,38.0,5989.726,13.0,9.0,19.0,5888.966429,5989.726,1.0
1130,3.0,102.0,5991.168,13.0,13.0,20.0,5890.837143,5991.168,1.0
1131,3.0,294.0,5993.070,13.0,17.0,21.0,5892.707857,5993.070,1.0
1132,3.0,358.0,5995.568,13.0,25.0,23.0,5896.449286,5995.568,1.0


In [8]:
wafer_info=get_pad_to_wafer(waferfile,dark_bias_lines=dark_bias_lines) #Load information about silicon assembly
smurf2det=smurf_to_detector(smurf2padloc,wafer_info)
smurf2det

,smurf_band,smurf_chan,smurf_freq,vna_freq,design_freq,index,mux_band,pad,mux_posn,biasline,pol,freq,det_row,det_col,rhomb,opt,det_x,det_y
1,4.0,335.0,4217.568,4217.568,4154.299286,1.0,1.0,3.0,16.0,8.0,A,150,6.0,9.0,B,1.0,42.809956,13.769804
2,4.0,399.0,4218.468,4218.468,4156.170000,2.0,1.0,7.0,16.0,8.0,A,150,4.0,9.0,B,1.0,37.509956,22.949673
3,4.0,47.0,4219.630,4219.630,4158.040714,3.0,1.0,11.0,16.0,8.0,A,150,2.0,9.0,B,1.0,32.209956,32.129542
4,4.0,111.0,4220.800,4220.800,4161.782143,5.0,1.0,19.0,16.0,8.0,A,150,10.0,10.0,B,1.0,56.059956,0.000000
5,4.0,175.0,4222.306,4222.306,4163.652857,6.0,1.0,23.0,16.0,8.0,A,150,8.0,10.0,B,1.0,50.759956,9.179869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,3.0,38.0,5989.726,5989.726,5888.966429,19.0,13.0,9.0,1.0,4.0,A,150,11.0,5.0,A,1.0,1.120022,53.139281
1130,3.0,102.0,5991.168,5991.168,5890.837143,20.0,13.0,13.0,1.0,4.0,A,150,9.0,5.0,A,1.0,-4.179978,43.959412
1131,3.0,294.0,5993.070,5993.070,5892.707857,21.0,13.0,17.0,1.0,4.0,A,150,7.0,5.0,A,1.0,-9.479978,34.779542
1132,3.0,358.0,5995.568,5995.568,5896.449286,23.0,13.0,25.0,1.0,4.0,A,150,3.0,5.0,A,1.0,-20.079978,16.419804


In [9]:
smurf2det.to_csv("MD14_Sv5_smurf2det.csv",index=False)

## Copper Assembly (change design_file)

In [19]:
#SMuRF channel assignments
chan_assign=glob.glob("Cv4/run1/*channel_assignment*.txt")
chan_assign.sort()
N_chan=chan_assign[4:]
S_chan=chan_assign[:4]
highband='N'
dark_bias_lines=[] #If certain sides are covered

In [20]:
N_vna_map="Cv4/run1/N_vna_map.csv"
S_vna_map="Cv4/run1/S_vna_map.csv"
posnfile="Cv4/band2posn.csv"
design_file="metadata/umux_32_map.pkl"
waferfile="metadata/copper_map_corrected.csv"

In [21]:
#South side VNA match
S_smurf2freq=all_smurf_chan_to_realized_freq(S_chan)
S_vna2pad=vna_freq_to_muxpad(S_vna_map,design_file)
S_smurf2mux=smurf_to_mux(S_smurf2freq,S_vna2pad,threshold=0.01)

#Repeat for north side VNA.
N_smurf2freq=all_smurf_chan_to_realized_freq(N_chan)
N_vna2pad=vna_freq_to_muxpad(N_vna_map,design_file)
N_smurf2mux=smurf_to_mux(N_smurf2freq,N_vna2pad,threshold=0.01)

smurf2mux=pd.concat([S_smurf2mux,N_smurf2mux],axis=0)

In [22]:
#All Smurf frequency to mux bondpad assignments
smurf2padloc=mux_band_to_mux_posn(smurf2mux,posnfile,highband='N')
smurf2padloc

,smurf_band,smurf_chan,smurf_freq,mux_band,pad,index,design_freq,vna_freq,mux_posn
0,0.0,185.0,4012.804,0.0,7.0,2.0,4022.741429,4012.804,17.0
1,0.0,121.0,4014.784,0.0,11.0,3.0,4024.612143,4014.784,17.0
2,0.0,249.0,4016.726,0.0,15.0,4.0,4026.482857,4016.726,17.0
3,0.0,505.0,4018.746,0.0,19.0,5.0,4028.353571,4018.746,17.0
4,0.0,261.0,4020.352,0.0,23.0,6.0,4030.224286,4020.352,17.0
...,...,...,...,...,...,...,...,...,...
1623,7.0,186.0,5973.606,13.0,46.0,61.0,5973.517143,5973.606,10.0
1624,7.0,122.0,5975.482,13.0,50.0,62.0,5975.387857,5975.482,10.0
1625,7.0,250.0,5977.920,13.0,54.0,63.0,5977.258571,5977.920,10.0
1626,7.0,6.0,5980.068,13.0,58.0,64.0,5979.129286,5980.068,10.0


In [23]:
wafer_info=get_pad_to_wafer(waferfile,dark_bias_lines=dark_bias_lines) #Load information about copper assembly
smurf2det=smurf_to_detector(smurf2padloc,wafer_info) #Complete map
smurf2det

,smurf_band,smurf_chan,smurf_freq,vna_freq,design_freq,index,mux_band,pad,mux_posn,biasline,pol,freq,det_row,det_col,rhomb,opt,det_x,det_y
0,0.0,185.0,4012.804,4012.804,4022.741429,2.0,0.0,7.0,17.0,11.0,A,150,8.0,0.0,C,1.0,-43.929978,-2.65
1,0.0,121.0,4014.784,4014.784,4024.612143,3.0,0.0,11.0,17.0,11.0,A,150,6.0,0.0,C,1.0,-33.329978,-2.65
2,0.0,249.0,4016.726,4016.726,4026.482857,4.0,0.0,15.0,17.0,11.0,A,150,4.0,0.0,C,1.0,-22.729978,-2.65
3,0.0,505.0,4018.746,4018.746,4028.353571,5.0,0.0,19.0,17.0,11.0,A,150,2.0,0.0,C,1.0,-12.129978,-2.65
4,0.0,261.0,4020.352,4020.352,4030.224286,6.0,0.0,23.0,17.0,11.0,A,150,0.0,0.0,C,1.0,-1.529978,-2.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1623,7.0,186.0,5973.606,5973.606,5973.517143,61.0,13.0,46.0,10.0,0.0,B,90,0.0,8.0,A,1.0,-43.929978,2.65
1624,7.0,122.0,5975.482,5975.482,5975.387857,62.0,13.0,50.0,10.0,0.0,B,90,0.0,6.0,A,1.0,-33.329978,2.65
1625,7.0,250.0,5977.920,5977.920,5977.258571,63.0,13.0,54.0,10.0,0.0,B,90,0.0,4.0,A,1.0,-22.729978,2.65
1626,7.0,6.0,5980.068,5980.068,5979.129286,64.0,13.0,58.0,10.0,0.0,B,90,0.0,2.0,A,1.0,-12.129978,2.65


In [12]:
smurf2det.to_csv("MD16_Cv4_smurf2det.csv",index=False)